In [1]:
import csv
import fasttext
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from sklearn.model_selection import KFold

In [2]:
def process_csv(df, output_name, output_folder):
    df.loc[:, 'text'] = '__label__' + df['category'].astype(str) + ' ' + df['review']
    output_file = f'{output_folder}/{output_name}.txt'
    df[['text']].to_csv(output_file, index=False, header=False, 
                quoting=csv.QUOTE_NONE,  quotechar="",  escapechar="\\")
    return output_file

In [3]:
def train_model(df, df_test, output_folder):
    output_file = process_csv(df, 'train', output_folder)
    output_file_test = process_csv(df_test, 'test', output_folder)
    model = fasttext.train_supervised(
        input=output_file, lr=0.1, epoch=20, wordNgrams=2)
    results = model.test(output_file_test)
    accuracy = results[1]
    return accuracy

In [4]:
def fastTextRun(data, corpus):
    X = corpus
    y = data['category']

    kf = KFold(n_splits=10)
    kf.get_n_splits(X)
    accuracy = []
    for train_index, test_index in kf.split(X, random_state=42):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        train_data = pd.concat([X_train, y_train], axis=1)
        test_data = pd.concat([X_test, y_test], axis=1)
        accuracy.append(train_model(train_data, test_data, "Output"))
    return accuracy

0.6205